### Logistic Regression

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/mnist', one_hot = True)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import examples.utils as utils
import time

In [3]:
tf.__version__

'1.9.0'

In [4]:
mnist_path = 'data/mnist'
utils.download_mnist(mnist_path)

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists


In [5]:
learning_rate = 0.01
batch_size = 128
epochs = 30
n_train = 60000
n_test = 10000

In [6]:
train,val,test = utils.read_mnist(mnist_path, flatten = True)
tf.reset_default_graph() #Useful to reset graph, because else tf throws
                         # errors with named entities

In [11]:
type(train)

tuple

In [9]:
np.shape(train[0]) #55k Images

(55000, 784)

In [10]:
np.shape(train[1]) # one-hot labels

(55000, 10)

In [12]:
'''
- Use tf to create the dataset
- Need to one set for training and test.
- Setting shuffle, shuffles the data
'''

train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000)
test_data = tf.data.Dataset.from_tensor_slices(test)

In [13]:
#Initilize the data with specific batch size
train_data = train_data.batch(batch_size)
test_data = test_data.batch(batch_size)

In [14]:
'''
- Next step:- Create iterator to get samples from the two datasets
- In linear regression-> used only train set. hence can create one iterator and draw samples from dataset.
- if train and test set and if we have one iterator for each date set, then need to build graph for each iterator
- better way: Create one signle iterator and initialize it with a dataset, when we need to draw from it.

Iterator represents thse state of iterating through a dataset
Iterator.from_structure :- creates a new uninitialzed iterator with a 
 given structure
 
 This iterator constructing method can be used to create an iterator that
 is reusable with many different datasets
 
 To initialize it with particular dataset:- Iterator.make_initializer(dataset)
'''
iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
img,label = iterator.get_next()

In [15]:
#Initialize the itertor with two different datasets
train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

In [16]:
#Note the shapes are different from Andrew's class
# X = img = (n x d) = (n0 x n1) = 55000 x 784
# w = (n1 x n2)
# b = (1 x n2)
w = tf.get_variable(shape = (784,10), name = "weights", initializer = tf.random_normal_initializer(0,0.01))
b = tf.get_variable(shape = (1,10), name = "bias", initializer = tf.zeros_initializer())

In [17]:
logits = tf.matmul(img,w) + b

In [18]:
#loss
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = logits, name = "entropy")
loss = tf.reduce_mean(entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [19]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [20]:
#Test set accuracty
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds,1),tf.argmax(label,1))
'''
tf.argmax(input, axis)
axis 1 -> index of max element in each row.
tf.argmax(preds,1):- index of max (highest logit)
tf.argmax(label,1):- one hot encoded hence, only 1 entry will have label 1
'''

accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

In [21]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    
    for i in range(epochs):
        sess.run(train_init) #get samples from the data
        total_loss = 0
        n_batches = 0
        
        try:
            while True:
                _,l = sess.run([optimizer,loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        print("Average loss for epoch {0} is {1}".format(i, total_loss/n_batches))
    print("Total time: {0}".format(time.time()-start_time))
    
    sess.run(test_init)
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass
    
    print("Accuracty:{0}".format(total_correct_preds/n_test))
    

Average loss for epoch 0 is 0.36646836445082065
Average loss for epoch 1 is 0.29454639375556346
Average loss for epoch 2 is 0.28577209749194077
Average loss for epoch 3 is 0.2796337234419446
Average loss for epoch 4 is 0.2744848436740942
Average loss for epoch 5 is 0.27121177289721576
Average loss for epoch 6 is 0.2698063459680524
Average loss for epoch 7 is 0.26876733174850775
Average loss for epoch 8 is 0.26669259085211644
Average loss for epoch 9 is 0.2680856530402982
Average loss for epoch 10 is 0.2629014372479084
Average loss for epoch 11 is 0.2598228190527406
Average loss for epoch 12 is 0.26267928609321284
Average loss for epoch 13 is 0.26255136738682905
Average loss for epoch 14 is 0.2599228248346684
Average loss for epoch 15 is 0.2594548969767814
Average loss for epoch 16 is 0.2617419119664403
Average loss for epoch 17 is 0.2582778943139453
Average loss for epoch 18 is 0.25696964998577915
Average loss for epoch 19 is 0.25833764824756356
Average loss for epoch 20 is 0.257023386